In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Caminho do arquivo Excel com a planilha "POTES"
excel_path = r'c:\Users\lucca.peixoto\Desktop\Pesquisas\Pesquisa Sorvete.xlsx'

# Caminho para salvar o arquivo processado
output_path = r'c:\Users\lucca.peixoto\Desktop\Pesquisas\Pesquisa Sorvete 13-11.xlsx'

# Nome da planilha e da coluna EAN
tabela_nome = "POTES"
coluna_ean = "CÓDIGO EAN"

# Inicializar o navegador (ChromeDriver)
driver = webdriver.Chrome()

# Função para buscar o EAN no site Economiza Alagoas
def buscar_ean_no_site(ean):
    driver.get("https://economizaalagoas.sefaz.al.gov.br/")
    time.sleep(2)  # Aguardar carregamento da página

    # Encontrar o campo de busca de EAN e digitar o EAN
    search_box = driver.find_element(By.ID, 'textoConsulta')
    search_box.clear()
    search_box.send_keys(ean)
    search_box.send_keys(Keys.RETURN)

    time.sleep(3)  # Aguardar os resultados carregarem
    
    try:
        # Encontrar o item da lista e clicar para abrir os resultados
        item_lista = driver.find_element(By.XPATH, "//li[@class='mdl-list__item mdl-list__item--two-line']")
        item_lista.click()
        time.sleep(3)  # Aguardar os detalhes da categoria carregarem

        # Capturar os preços dos produtos
        cartoes = driver.find_elements(By.CLASS_NAME, 'cartao')
        
        resultados = {}

        for cartao in cartoes:
            try:
                # Capturar nome do supermercado
                nome_supermercado = cartao.find_element(By.CLASS_NAME, 'cartao_contribuinte_bloco_esquerdo').text.strip().split('\n')[0]
                
                # Capturar o preço mais recente usando o elemento de tempo informado
                tempo_elemento = cartao.find_element(By.XPATH, ".//span[contains(text(),'Há')]")
                preco_elemento = tempo_elemento.find_element(By.XPATH, "../span[@class='valor_ultima_venda']")
                preco = preco_elemento.text.strip()
                
                # Verificar se o supermercado está nas colunas da tabela
                for supermercado in tabela.columns[2:]:  # Considera as colunas a partir da terceira (2º índice)
                    if nome_supermercado in supermercado:
                        resultados[supermercado] = preco
                        break
            except Exception as e:
                print(f"Erro ao capturar dados do cartão: {str(e)}")

        return resultados

    except Exception as e:
        print(f"Erro ao buscar EAN {ean}: {str(e)}")
        return None

# Abrir o arquivo de saída Excel para gravar os resultados processados
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Carregar a planilha "POTES"
    tabela = pd.read_excel(excel_path, sheet_name=tabela_nome, dtype=str)
    
    # Verificar se a tabela foi carregada corretamente
    if tabela.empty:
        print(f"A planilha {tabela_nome} está vazia ou não foi encontrada.")
    else:
        # Iterar sobre os produtos na tabela e buscar preços no site
        for index, row in tabela.iterrows():
            ean = row[coluna_ean]
            
            # Verificar se o EAN está vazio ou NaN e continuar para o próximo
            if pd.isna(ean) or ean.strip() == '':
                continue  # Pular para o próximo EAN

            ean = ean.split('.')[0]  # Garantir que não tenha caracteres extra
            resultados = buscar_ean_no_site(ean)
            
            # Preencher resultados ou atribuir "0" nos preços
            if resultados is not None:
                for supermercado in tabela.columns[2:]:  # Colunas de supermercado
                    valor = resultados.get(supermercado, '0')
                    tabela.at[index, supermercado] = valor
                    print(f"{tabela_nome} - EAN: {ean} | Supermercado: {supermercado} | Valor: {valor}")
            else:
                print(f"{tabela_nome} - EAN: {ean} | Nenhum resultado encontrado. Definindo preços como 0.")
                for supermercado in tabela.columns[2:]:
                    tabela.at[index, supermercado] = '0'

        # Salvar a aba processada no arquivo de saída
        tabela.to_excel(writer, sheet_name=tabela_nome, index=False)

driver.quit()

print(f"Script finalizado. Planilha salva em {output_path}.")


POTES - EAN: 7896744002614 | Supermercado: SENDAS DISTRIBUIDORA S/A | Valor: R$ 24,77
POTES - EAN: 7896744002614 | Supermercado: MIX MATEUS | Valor: R$ 26,89
POTES - EAN: 7896744002614 | Supermercado: UNI COMPRA SUPERMERCADOS LTDA | Valor: R$ 26,99
POTES - EAN: 7896744002614 | Supermercado: SAM'S CLUB | Valor: R$ 27,98
POTES - EAN: 7896744002614 | Supermercado: SUPERMERCADOS BOMPRECO | Valor: R$ 31,99
POTES - EAN: 7896744002614 | Supermercado: BOMPRECO SUPERMERCADOS DO NORDESTE LTDA | Valor: R$ 31,99
POTES - EAN: 7896744002614 | Supermercado: G BARBOSA | Valor: R$ 31,99
POTES - EAN: 7896744002614 | Supermercado: ESPECIARYA INDUSTRIA E COMERCIO LTDA | Valor: R$ 27,90
POTES - EAN: 7896744002614 | Supermercado: SUPERMERCADO PONTO CERTO | Valor: R$ 27,99
POTES - EAN: 7896744002614 | Supermercado: CESTA DE ALIMENTOS | Valor: R$ 25,99
POTES - EAN: 7896744002614 | Supermercado: SUPER GIRO | Valor: R$ 28,99
POTES - EAN: 7896744002614 | Supermercado: SUPERMERCADO SAO DOMINGOS | Valor: R$ 27,99
